# Data Generation Cleanup

- [ ] Check how many actions per franka, and match their positions
  - [ ] Convert EEF to Actions, and see how they match each other
- [ ] Convert EEF Targets to Actions
- [ ] Execute a trajectory (EEF to ACtion converted vs direct actions from the dataset)
- [ ] Reset to random environment, transform trajectory and execute
- [ ] 

In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
    """
    Retrieves env metadata from dataset.

    Args:
        dataset_path (str): path to dataset

    Returns:
        env_meta (dict): environment metadata. Contains 3 keys:

            :`'env_name'`: name of environment
            :`'type'`: type of environment, should be a value in EB.EnvType
            :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
    """
    dataset_path = os.path.expanduser(dataset_path)
    f = h5py.File(dataset_path, "r")
    if ds_format == "robomimic":
        env_meta = json.loads(f["data"].attrs["env_args"])
    else:
        raise ValueError
    f.close()
    return env_meta

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [17]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

In [56]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [57]:
data['data']['demo_10']['datagen_info'].keys()

<KeysViewHDF5 ['eef_pose', 'gripper_action', 'object_poses', 'subtask_term_signals', 'target_pose']>

In [58]:
eef_poses.shape, gripper_actions.shape

((238, 2, 4, 4), (238, 2, 6))

In [59]:
action_trajectory.shape

(238, 24)

In [60]:
action_trajectory[0]

array([ 4.25097525e-02,  9.12558692e-03,  4.32557883e-02, -3.03393696e-04,
       -2.73457170e-02, -8.96739494e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.57079633e+00,
        5.20099715e-02,  1.32346484e-01,  4.79033702e-02,  1.38189155e-03,
       -1.03843641e-02,  3.85016240e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  7.93970488e-02,  1.57079633e+00])

In [61]:
target_eef_poses[0] - eef_poses[0]
p0 = PoseUtils.unmake_pose(eef_poses[0])
p1 = PoseUtils.unmake_pose(target_eef_poses[0])
p0, p1

((array([[-0.12154334, -0.24502211,  1.09128778],
         [-0.12433157,  0.22114966,  1.10033209]]),
  array([[[-0.99707392, -0.07581728, -0.00976386],
          [-0.00952934, -0.00345441,  0.99994863],
          [-0.07584711,  0.99711574,  0.00272182]],
  
         [[ 0.98572615, -0.12634151, -0.11127346],
          [-0.11088725,  0.01010797, -0.99378159],
          [ 0.12668062,  0.99193531, -0.00404597]]])),
 (array([[-0.11941785, -0.24456583,  1.09345057],
         [-0.12173107,  0.22776699,  1.10272726]]),
  array([[[-0.99597645, -0.08945754, -0.00531572],
          [-0.00507369, -0.00293271,  0.99998283],
          [-0.08947159,  0.99598632,  0.00246703]],
  
         [[ 0.98700808, -0.13165436, -0.09209875],
          [-0.09198753,  0.00693941, -0.99573598],
          [ 0.13173209,  0.99127139, -0.00526131]]])))

In [62]:
p0[0][0]

array([-0.12154334, -0.24502211,  1.09128778])

In [63]:
obfull = env.reset()

In [64]:
obfull['robot0_eef_pos'], obfull['robot0_eef_quat']

(array([-0.11378872, -0.27246925,  1.10466974]),
 array([ 0.99748611, -0.02861244,  0.06460603,  0.00537085]))

In [21]:
delta_pose =  p1[0] - p0[0]
print("Delta pose: ", delta_pose)
max_dpos = env.robots[0].controller.output_max[0]
delta_pose = np.clip(delta_pose/max_dpos, -1., 1.)

Delta pose:  [[ 0.00376664 -0.00140738  0.00365097]
 [-0.00383368 -0.00023321 -0.00131807]]


AttributeError: 'FixedBaseRobot' object has no attribute 'controller'

In [26]:
scaling_factor = delta_pose[0]/np.array([7.53327273e-02, -2.81475012e-02,  7.30194213e-02])

In [27]:
scaling_factor

array([0.05, 0.05, 0.05])

In [28]:
scaling_factor2 = delta_pose[1]/np.array([-7.66736512e-02, -4.66412965e-03, -2.63613784e-02])

In [29]:
scaling_factor2

array([0.05, 0.05, 0.05])

In [70]:
target_rot = p1[1][1]
curr_rot = p0[1][1]

In [71]:
# target_rot.shape

In [72]:
# delta_rot_mat.shape

In [73]:
# normalized delta rotation action
delta_rot_mat = target_rot.dot(curr_rot.T)
delta_quat = T.mat2quat(delta_rot_mat)
delta_rotation = T.quat2axisangle(delta_quat)
# delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
# return np.concatenate([delta_position, delta_rotation])
delta_rotation

array([ 0.00069094, -0.00519254,  0.01925058], dtype=float32)

In [ ]:
# action_trajectory.shape

(238, 24)

In [76]:
delta_rotation/action_trajectory[0][15:18]

array([0.4999936 , 0.50003444, 0.49999405])

In [ ]:
delta_rotation

### Clean

max_dpos = 0.05
delta_pos = np.clip(delta_position / max_dpos, -1., 1.)


max_drot = 0.5 
delta_rot_mat = target_rot.dot(curr_rot.T)
delta_quat = T.mat2quat(delta_rot_mat)
delta_rotation = T.quat2axisangle(delta_quat)
delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)

In [ ]:
for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]